## Ortho Prediction Software Developing by krish:
#### Check all Function Working better or Not

#### main / Global Variable use in this program

In [1]:
import time , os, sys,subprocess , numpy as np , pandas as pd

In [2]:
#Blastp = /usr/bin/blastp
Blastp = "blastp"
Blastp_data =  './blastp_data/'
cpu_count = 1 # Default value
blastp_matrix = "BLOSUM62" 
genome = ""
infinite_loop = 70
inflation_factor = 1.4
#mode = None
mode = None
Cluster_out = "./cluster_out/"
#threshold_score : 0
threshold_score =0
Save_raw_blastp_score = False
Score_file = "./score_file"
Species = "./species/"
species = "./species/"
verbose = False

In [3]:
import glob
a = glob.glob("species/""*")
a[1].split("\\")[1]

'CAC'

## 1. GetMatrixNumber()

In [4]:
#The Following matrix files must be in Folder
import os
def GetMatrixNumber():
    "This Function will Return the Matrix name choosed by User"
    import sys
    print ("\n1. BLOSUM45\n2. BLOSUM62\n3. BLOSUM82\n4. Quit")

    metrix_num = input("\nEnter a matrix number: ")
    if metrix_num not in ("1","2",'3'):
        print("Wrong input *%s*Sorry not in list\n"%metrix_num,"*"*20,"Good Bye","*"*20,"\n")
        sys.exit(1)
    if metrix_num =="1":
        return "BLOSUM45"
    if metrix_num == "2":
        return "BLOSUM62"
    if metrix_num == "3":
        return "BLOSUM82"
#GetMatrixNumber()

### 2. GetQuerySequence(genome)

In [5]:

def GetQuerySequence(genome):
    """This Function Read Fastaq Files and return as a list Format with gene Position as a index
    Later I will reduce code Lengths and Speed"""
    global species
    gene_sequence=""
    gene_sequence_list= []
    try:
        with open(species+genome) as gene:
            for each_line in gene:
                if ">" in each_line:
                    if gene_sequence != "":                        
                        gene_sequence_list.append(gene_sequence)
                        gene_sequence = ""
                gene_sequence = gene_sequence+each_line
            gene_sequence_list.append(gene_sequence)                   
        return gene_sequence_list                                                                
    except IOError as err:
        print ("IOError occurred in GetQuerySequence function : " + str(err))

In [6]:
b = GetQuerySequence("AAE")
#species

In [7]:
b[10]

'>gi|15605623|ref|NP_212996.1| 50S ribosomal protein L16 [Aquifex aeolicus VF5]\nMYLQPKKVRWRKQRRGTLKGKAFRGNKLAFGEYGIQALDRAWITQQQIEAVRVALVRSLKKGAKVWIRIF\nPDKPYTKKPNEVRMGGGKGDPEGFVAVVKPGRILFEFTGVPEEVAQEAFTIAASKLPIPVRLVKYGEFTF\nKY\n'

## 3.Write Query(query, parallel_num)

In [8]:
def WriteQuery(query, parallel_num):
    # query is j , part of Genome , gene id , a single sequence
    try:
        with open("./query"+"_"+str(parallel_num)+".csv", "w") as write_query:
            write_query.write(query)
    except IOError as err:
        print ("IOError occurred in WriteQuery function : " + str(err))

In [9]:
WriteQuery("hello",5)
#hello is is query name and 

## 4. RunBlast( Subject , parallel_num)

In [10]:
import subprocess
def RunBlast(subject, parallel_num):
    """By this Function it will create a Pipe line to run Blastp in Computer by input Parameter
     subject is a small size gene passed not a whole
     parallel_nu is the CPU selected by user"""
    subject = Species+subject 
    Blastp = "blastp"
    cmd = [Blastp, "-query","./query"+"_"+str(parallel_num), "-subject", subject,"-matrix", blastp_matrix,
           "-outfmt", "10 qseqid sseqid score length"]
    
    run_blastp =subprocess.Popen(cmd,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #qseqid = query (e.g. gene sequence id) , sseqid = subject(e.g. reference genome)
    #sequence id score = Rawscore length = Alignment length
    
    run_blastp_stream = run_blastp.communicate()
    run_blastp_output_stream = run_blastp_stream[0]
    run_blastp_error_stream = run_blastp_stream[1]  
    
    #display(run_blastp)
    #display(run_blastp_error_stream)
    return run_blastp_output_stream
#RunBlast("AAE","AAE")

In [11]:
RunBlast("AAE",2)

b'NP_001267053.1,gi|15607130|ref|NP_214512.1|,174,519\r\nNP_001267053.1,gi|15606768|ref|NP_214148.1|,59,33\r\nNP_001267053.1,gi|15606987|ref|NP_214369.1|,55,57\r\nNP_001267053.1,gi|15605766|ref|NP_213143.1|,55,39\r\nNP_001267053.1,gi|15606556|ref|NP_213936.1|,52,26\r\nNP_001267053.1,gi|15605625|ref|NP_212999.1|,51,44\r\nNP_001267053.1,gi|15606023|ref|NP_213400.1|,52,61\r\nNP_001267053.1,gi|15606308|ref|NP_213687.1|,52,33\r\n'

## Get_Same_Species_Forward_Best_Hit(blastp_score):

In [38]:
def Get_Same_Species_Forward_Best_Hit(blastp_score):
    blastp_score_split_list  = []
    temp_best_score = ["-1",'-1','-1']
    second_temp_best_score = []
    best_score = []
    second_best_score = []
    blastp_score_split = blastp_score.split('\n')
    
    del blastp_score_split[-1] #delete of [''] in last index 
    for i in blastp_score_split:
        blastp_score_element = i.split(",")
        blastp_score_split_list.append(blastp_score_element)
    for k in blastp_score_split_list: #ex) k in ['gi|15605613|ref|NP_212986.1|', 'gi|15605613|ref|NP_212986.1|', '3702', '699']
        if k[0] == k[1]:
            best_score.append(k)
        elif k[0] !=k[1]:
            print(k[2], temp_best_score[2])
            if int(k[2]) >int(temp_best_score[2]): #Compare the Score , Greater than
                temp_best_score = k
            elif int(k[2]) == int(temp_best_score[2]): # Is equal
                if int(k[3])> int(temp_best_score[3]) : #Compare the length
                    temp_best_score = k
                elif int(k[3]) == int(temp_best_score[3]):
                    second_temp_best_score.append(k)
        #print( "################ temp Best Score #####################", temp_best_score)
        second_best_score.append(temp_best_score)
        for j in second_temp_best_score:
            if j[2] == temp_best_score[2] and j[3] == temp_best_score[3]:
                second_best_score.append(j)
        for m in second_best_score:
            if (best_score[0][2] == m[2] and int(best_score[0][3])<= int(m[3])) or int(best_score[0][2])< int(m[2]): 
                #'104'<"23" is True because of string. So the int Function is Used.
                best_score.append(m)
    return best_score
                        

In [12]:
Get_Same_Species_Forward_Best_Hit(blastp_score.decode())

NameError: name 'Get_Same_Species_Forward_Best_Hit' is not defined

In [14]:
blastp_score.decode()

NameError: name 'blastp_score' is not defined

## 21. Read_Species_List()

In [39]:
import glob
def Read_Species_List(pr=0):
    """ If pr is 1, it will print "Species_List". """ 
    global species
    read_species =glob.glob(species+"*")   
    selected_species_dic = {}
    backward_selected_species_dic = {}
    #number = 0
    for i, spec in enumerate(sorted(read_species), start=1):
        #variable name spec is required because local Variable and global Variable can cause Conflict
        selected_species_dic[i] = spec.split('\\')[-1]
        backward_selected_species_dic[spec.split('\\')[-1]] = i 
        if pr == 1 :
            print (str(i)+".", spec.split('\\')[-1])
        number = i
    return selected_species_dic, backward_selected_species_dic, number

In [14]:
def read_species(pr =1):
    import os
    read_species = os.listdir("species/")
    for i,species in enumerate(read_species,start =1):
        print(i,species)
#read_species()

## 22. Del_File(path,file)

In [15]:
def Del_File(path,file):
    import subprocess
    
    del_file = subprocess.Popen(["del "+path+file],stdout=subprocess.PIPE, stderr = subprocess.PIPE,shell= True)
    del_file_stream = del_file.communicate()
    if not del_file_stream[1]:
        print("Done to del"+path+file)
    elif del_file_stream[1]:
        print(del_file_stream[1])

#### i developed the below Function to remove the file may be different but will check later 

In [16]:
def del_(file):
    import os
    try:
        os.remove(file)
        print("Successfully_remove",file)
    except:
        print("Check the File And Path")

In [22]:
del_("krish")

Check the File And Path


### 23.Check_File(file)

In [17]:
def Check_File(file):
    "Check the file weather exist or not"
    file_list = glob.glob(file+'*')    
    if (Cluster_out+"_geneID_S"+str(threshold_score)+"_"+str(inflation_factor) or Cluster_out+"_KO_ID_S"+str(threshold_score)+"_"+str(inflation_factor)) in file_list:
        print ("Please, set other name of output.")
        sys.exit(2)

### Oneway_Threshold_Best_Hit(mode)
modes are 1. Blastp 2. BLASTP using precalculated data 3. Clustering 

In [40]:
def Oneway_Threshold_Best_Hit(mode):
    "This Function accept the mode and Run Program and return backward_best_hit_work_list"
    print("Oneway_Threshold_Best_Hit Is running")
    process_list = []
    backward_best_hit_work_list = [] # Container to sava backward bect hit work
    if "1" in mode:
        print("Blastp Mode is Selected")
        for i in user_selected_number: # user_selected_number is the list of Species 1,2 ..
            queryV =  GetQuerySequence(selected_species_dic[i]) #select the user select number 
            #queryV is the list type 
            for k in user_selected_number:
                if k <i:
                    print(k)
                    continue
                else:
                    print("Doing the blastp & Forward best hit searches between %s genome and %s genome"%\
                          (selected_species_dic[i],selected_species_dic[k]))
                    queryV_len = len(queryV) #Repeating Time
                    
                    if cpu_count ==1:
                        blastp_time_start = time.time() #start Time
                        print(i,k, cpu_count) # 2 2 1 
                        RunParallelQuery(i , k , queryV , cpu_count)
                        blastp_time_end = time.time()
                        print("The Blastp & Forward best hit searches took %.2f minutes"%((blastp_time_end- blastp_time_start)/60))
                    else:
                        if queryV_len < cpu_count : 
                            # If the number of queryV_len is less than cpu_count, Remark will select the number of queryV_len
                            blastp_time_start = time.time()
                            parallel_query = DivisionParallelQuery(queryV, 1, queryV_len, queryV_len)
                            #1 is query_division_value. Because queryV_len / queryV_len( = cpu_count) is 1
                            for m in range(queryV_len):
                                process = multiprocessing.Process(target = RunParallelQuery, args = (i,k, parallel_query[m],m+1))
                                #args(i => species of query, k = > species of subject , m+1 = > cpu_count ex) 1,2 ...)
                                process_list.append(process)
                                process.start()
                            for n in process_list:
                                n.join()
                            blastp_time_end = time.time()
                            print("The blastp & forward best hit searches took %.2f minutes"%((blastp_time_end - blastp_time_start)/60))
                        else:
                            blastp_time_start = time.time()
                            query_division_value = queryV_len / cpu_count
                            parallel_query = DivisionParallelQuery(queryV, query_division_value , cpu_count, queryV_len)
                            
                            for m in range(cpu_count):
                                process = multiprocessing.Process(target=RunParallelQuery, args= (i,k,parallel_query[m],m+1))
                                #args(i =>species of query , k = > species of subject , m+1 => cpu_count ex)1,2 ...
                                process_list.append(process)
                                process.start()
                            for n in process_list:
                                n.join()
                            blastp_time_end = time.time()
                            print("The blastp & forward best hit searches took %.2f minutes"%((blastp_time_end - blastp_time_start)/60))
                            
                    if not i == k:
                        backward_best_hit_work_list.append((i,k,queryV_len))
    elif "2" in mode:
        print("BLASTP using precalculated data mode Running")
        for i in user_selected_number: #Select species to write query
            queryV = GetQuerySequence(selected_species_dic[i])
            for k in user_selected_number: #Select of subject
                if Blastp_data + selected_species_dic[i]+"_"+selected_species_dic[k] + "_oneway_threshold_best_hit_S"+\
                str(threshold_score) in precalculated_data_list:
                    used_precalculated_data_list.append(selected_species_dic[i]+"_"+selected_species_dic[k])
                    continue
                else:
                    if k<i: # gene ===> query 1 -> 1 , 1 ->2  , 1 ->3 ,2 ->2 , 2->3
                        continue
                    else:
                        print("Doing the blastp & Forward best hit searches between %s genome and %s genome"%(selected_species_dic[i],selected_species_dic[k]))
                        queryV_len = len(queryV)
                                    
                        if cpu_count ==1:
                            blastp_time_start = time.time()
                            RunParallelQuery(i,k,queryV, c)
                            blastp_time_end = time.time()
                            print("The blastp & forward best hit searches took %.2f minutes"%\
                                  ((blastp_time_end-blastp_time_start)/60)) 
                        else:
                            if queryV_len<cpu_count:
                                #if the number of queryV_len is less than cpu_count, Remark will select the number of queryV_len.
                                blastp_time_start = time.time()
                                parallel_query = DivisionParallelQuery(queryV,1,queryV_len, queryV_len)
                                #1 is query division_value. Because queryV_len /queryV_len( = cpu_count) is 1
                                for m in range(queryV_len):
                                    process = multiprocessing.Process(target= RunParallelQuery, args = (i,k,parallel_query[m],m+1))
                                    #args (i => species of query, k = > species of subject, m+1 = > cpu_count e.x) 1,2,3...)
                                    process_list.append(process)
                                    process.start()
                                    for n in process_list:
                                        n.join()
                                    blastp_time_end = time.time()
                                    print("The blastp & forward best hit searches took %.2f minutes"%\
                                          ((blsatp_time_end- blastp_time_start)/60))
                                else:
                                    blastp_time_start = time.time()
                                    query_division_value = queryV_len / cpu_count
                                    parallel_query = DivisionParallelQuery(queryV , query_division_value, cpu_count , queryV_len)
                                    
                                    for m in range(cpu_count):
                                        process = multiprocessing.Process(target = RunParallelQuery, args = (i,k,parallel_query[m],m+1))
                                        #args (i = > species of query, k = > species of subject, m+1 = > cpu_count ex ) 1,2,...
                                        process_list.append(process)
                                        process.start()
                                    for n in process_list:
                                        n.join()
                                    blastp_time_end = time.time()
                                    print("The blastp & forward best hit searches took %.2f minutes"%((blastp_time_end - blastp_time_start)/60))
                            new_calculated_data_list.append(selected_species_dic[i]+"_"+selected_species_dic[k])
                            if not i ==k :
                                backward_best_hit_work_list.append(i,k,queryV_len)
    return backward_best_hit_work_list

In [57]:
#Oneway_Threshold_Best_Hit("1")
queryV  = GetQuerySequence("AAE")

In [70]:
RunParallelQuery(1,1,queryV,1)

Parallel Query Running
Running Get Forward Best Hit
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>> Get Forward Best

TypeError: decoding to str: need a bytes-like object, int found

In [54]:
with open(Score_file+selected_species_dic[species_of_query]+"_" + selected_species_dic[species_of_subject]+"_oneway_threshold_best_hit_S"+str(threshold_score,"a")) as oneway_threshold_best_hit:
    oneway_threshold_best_hit.readline()
    

NameError: name 'species_of_query' is not defined

NameError: name 'i' is not defined

In [35]:
#Oneway_Threshold_Best_Hit("1")
quer = GetQuerySequence(selected_species_dic[11])  
type(quer)

list

In [26]:
Oneway_Threshold_Best_Hit("1")

NameError: name 'Oneway_Threshold_Best_Hit' is not defined

### Get_Same_Species_Forward_Best_Hit(blastp_score)

In [ ]:
def Get_Same_Species_Forward_Best_Hit(blastp_score):
    """ Search the forward best hit among the blastp scores of same species. Because there are an duplicated genes in a same 
    genome. When the blastp score compare with blastp score of duplicate gene, if score and length are same, blastp score of duplicated
    gene is added to a second best score."""
    print("Running Get_Same_Species_Forward_Best_Hit")
    blastp_score_split_list = []
    temp_best_score = ['-1','-1','-1']
    second_temp_best_score = []
    best_score = []
    second_best_score = []
    blastp_score_split  = blastp_score.split("\n")
    

### Run Parallel Query

In [49]:
def RunParallelQuery(species_of_query, species_of_subject, queryV, parallel_num):
    #from progress import bar
    #i --> Species_of_query , k --> Species_of_subject , queryV is Genome list whole file (i , k are part of this)
    """This Function helps to run operation on Several Process
    Run the Following Functions. WriteQuery , RunBlast , Get_Same_Species_Forward_Best_Hit, GetForwardBestHit"""
    
    print("Parallel Query Running") 
    #bar = Bar("processing"+str(cpu_count),max = len(queryV))
    #bar = bar("processing"+str(cpu_count),max = len(queryV))  #python 3
    for j in queryV:
        #bar.next()
        WriteQuery(j, cpu_count)
        #WriteQuery() Function Only write file with info
        blastp_score= RunBlast(selected_species_dic[species_of_subject],parallel_num)
        
        if blastp_score != "":
            best_score, blastp_score_split_list = GetForwardBestHit(blastp_score.decode()) #decode() will change to str Format
            if species_of_query == species_of_subject: # e.g. AAE == AAE It will save best_score without reversing RunBlast.
                same_species_forward_best_score = Get_Same_Species_Forward_Best_Hit(blastp_score.decode())
                
                for best_score_element in same_species_forward_best_score:
                    if best_score_element[0] == best_score_element[1]: #ex [A1 of AAE, A! of AAE , 30 ]
                        with open(Score_file+selected_species_dic[species_of_query]+"_" + selected_species_dic[species_of_subject]+"_oneway_threshold_best_hit_S"+str(threshold_score,"a")) as oneway_threshold_best_hit:
                                  save_best_score= selected_species_dic[species_of_query]+"_"+best_score_element[0].split("\s"[0])+" "+selected_species_dic[species_of_subject]+ "_"+best_score_element[1].split("\s")[0]+" "+best_score_element[2]+"\n"
                                  oneway_threshold_best_hit.write(save_best_score)
                                 #best_score_element[0].split("|" ==> ["gi", '15642790','ref', "NP_227831.1",""])
                    else:
                        # ex ) [ A1 of AAE , A2 of AAE, 30]
                        with open (Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+
                                  "_second_oneway_threshold_best_hit_S"+ str(threshold_score),"a") as second_onewaythreshold_best_hit:
                                  second_save_best_score = selected_species_dic[species_of_query]+"_"+best_score_element[0].split("\s")[0].split("\s")[0]+\
                                  selected_species_dic[species_of_subject]+"_"+best_score_element[1].split("\s")[0]+" "+best_score_element[2]+" "+best_score_element[3]+"\n"
                                  second_oneway_threshold_best_hit.write(second_save_best_score)
            
                else: # If species_of_query not equal with species_of_subject , run reversing RunBlast
                    for best_score_element in best_score:
                        if not "-1" in best_score_element :
                            with open(Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+"_S"+str(threshold_score)+"_"+str(parallel_num),"a") as save_blastp:
                                save_blastp_score_split_list.write(blastp_score_split_list_save)
            #bar.finish()
            return 

In [50]:
blastp_score= RunBlast(selected_species_dic[1],1)

In [78]:
RunBlast("AAE",1)

b'gi|15004706|ref|NP_149166.1|,gi|15606796|ref|NP_214176.1|,166,240\r\ngi|15004706|ref|NP_149166.1|,gi|15606469|ref|NP_213849.1|,65,186\r\ngi|15004706|ref|NP_149166.1|,gi|15606745|ref|NP_214125.1|,55,117\r\ngi|15004706|ref|NP_149166.1|,gi|15606205|ref|NP_213582.1|,51,61\r\ngi|15004706|ref|NP_149166.1|,gi|15605883|ref|NP_213260.1|,51,23\r\ngi|15004706|ref|NP_149166.1|,gi|15606235|ref|NP_213613.1|,49,16\r\ngi|15004706|ref|NP_149166.1|,gi|15606075|ref|NP_213452.1|,48,86\r\ngi|15004706|ref|NP_149166.1|,gi|15605704|ref|NP_213081.1|,49,36\r\ngi|15004706|ref|NP_149166.1|,gi|15606425|ref|NP_213805.1|,49,64\r\ngi|15004706|ref|NP_149166.1|,gi|15605876|ref|NP_213253.1|,47,30\r\ngi|15004706|ref|NP_149166.1|,gi|15605857|ref|NP_213234.1|,48,44\r\ngi|15004706|ref|NP_149166.1|,gi|15606768|ref|NP_214148.1|,47,98\r\ngi|15004706|ref|NP_149166.1|,gi|15606894|ref|NP_214275.1|,48,72\r\n'

### GetQuerySequence (genome)

In [21]:
def GetQuerySequence(genome):
    """This Function Read Fastaq Files and return as a list Format with gene Position as a index
    Later I will reduce code Lengths and Speed
    """
    gene_sequence=""
    gene_sequence_list= []
    try:
        with open(Species+genome) as gene:
            for each_line in gene:
                if ">" in each_line:
                    if gene_sequence != "":                        
                        gene_sequence_list.append(gene_sequence)
                        gene_sequence = ""
                gene_sequence = gene_sequence+each_line
            gene_sequence_list.append(gene_sequence)                   
            return gene_sequence_list                        
                                            
    except IOError as err:
        print ("IOError occurred in GetQuerySequence function : " + str(err)    )

## GetForwardBestHit(blastp_score)

In [22]:
def GetForwardBestHit(blastp_score):
    "Search the forward best hit among blastp scores of same species"
    print("Running Get Forward Best Hit")
    blastp_score_split_list = []
    temp_best_score = ['-1','-1','-1']
    second_temp_best_score = []
    best_score = []
    blastp_score_split = blastp_score.split("\n")
    
    del blastp_score_split[-1] #Remove last index of "\n"
    for i in blastp_score_split:
        blastp_score_element = i.split(",")
        blastp_score_split_list.append(blastp_score_element)
    for k in blastp_score_split_list:
        # ex) k is ["gi|15605613|ref|NP_212986.1|",'gi|15605613|ref|NP_212986.1|', '3702', '699']
        print(">>>>>>>>>>>>>>>> Get Forward Best Hit>>>>>>>>>>>>>>>>>>>>>>>")
        if int(k[2])>int(temp_best_score[2]): # Compare Score
            temp_best_score = k
        elif int(k[2]) == int(temp_best_score[2]):
            if int(k[3])> int(temp_best_score[3]): #Comapare the Length
                temp_best_score = k
            elif int(k[3]) == int(temp_best_score[3]):
                second_temp_best_score.append(k)
    print("## temp best Score ###",temp_best_score)
    best_score.append(temp_best_score)
    for j in second_temp_best_score:
        if j[2] == temp_best_score[2] and j[3] == temp_best_score[3]:
            best_score.append(j)
    return best_score , blastp_score_split_list

In [80]:
#a = a.split("\n")
print(a[-1])
#del(a[-1])

gi|15605613|ref|NP_212986.1|,gi|15606309|ref|NP_213688.1|,52,166


In [89]:
GetForwardBestHit(str(blastp_score))

Running Get Forward Best Hit
## temp best Score ### ['-1', '-1', '-1']


([['-1', '-1', '-1']], [])

### DivisionParallelQuery(queryV,query_division_value, cpu_count , queryV_len)

In [83]:
def DivisionParallelQuery( queryV ,query_division_value,cpu_count,queryV_len):
    """ Running Division parallel Query,input is species_of_query , species_of_subject, queryV, parallel_num"""
    parallel_query = []
    parallel_query_start = 0
    if queryV_len % cpu_count == 0: #Perfect division by CPU Count
        for i in range(cpu_count):
            i+=1
            if parallel_query_start ==0:
                parallel_query.append(queryV[int(parallel_query_start):i*int(query_division_value)])
                parallel_query_start +=1
            else:
                parallel_query.append(queryV[int(parallel_query_start)*int(query_division_value):i*int(query_division_value)])
                parallel_query_start +=1
    else: # If the division is imperfect
        for i in range(cpu_count):
            i += 1
            if parallel_query_start == 0:
                parallel_query.append(queryV[int(parallel_query_start):i*int(query_division_value)])
                parallel_query_start +=1
            elif i<cpu_count:
                parallel_query.append(queryV[int(parallel_query_start)*int(query_division_value):])
                parallel_query_start +=1
        return parallel_query

In [105]:
DivisionParallelQuery(queryV ="My name is krishna n i am From Nepal" ,query_division_value=10,cpu_count=2,queryV_len=5)

['My name is']

## Main Program Testing

In [23]:
# arg parse is not done here
# If parameter not Provided the below code will run

In [27]:
Read_Species_List(pr=1) #Return values
print()

1. AAE
2. CAC
3. ECO
4. ECU
5. HIN
6. LLA
7. SCE
8. SPO
9. SPY
10. SYN
11. TMA
12. YPE



In [24]:
#if not sys.argv[1:]

import sys
import multiprocessing

# If Program not Passed with Parameter
 # if Not verified Conflict arise for local and global Variable
print ("1. BLASTP. \n2. BLASTP using precalculated data. \n3. Clustering.\n")
mode = input(">> Select a mode or modes (1 or 2 or 1 3 or 2 3): ") 
selected_species_dic, backward_selected_species_dic, number_i = Read_Species_List(pr=1)
selected_number = input(">> Select Genomes to detect Orthologs(e.g. 1 2 3 4 5 or 1-5) : ")


if selected_number.find("-")>0:
    SN = selected_number.split("-")
    if int(SN[-1])>number_i:
        print("Your input must be less than" , number_i)
        sys.exit(2)
    else:
        user_selected_number = range(int(SN[0]),int(SN[-1])+1)
        print(user_selected_number)
        for j in user_selected_number:
            print(selected_species_dic[j],end=" ")
        print("Are Selected")
else:
    user_selected_number = sorted(set([int(read_species) for read_species in selected_number.split()]))
    if int(user_selected_number[-1])>number_i:
        print ("\nWrongInput\nInput must be less than",number_i)
        sys.exit(2)
    else:
        for j in user_selected_number:
            print(selected_species_dic[j], end =" ")

blastp_matrix = GetMatrixNumber()
cpu_count = int(input("You can use %s processors. \nIf you input >2, The Program will run a parallel--"\
                      %multiprocessing.cpu_count()+"Enter the number of CPU you want to use :"))
if "3" in mode:
    inflation_factor = input("Enter the inflation factor to cluster")
    Cluster_out = input("Set the name of Clustering output :")
    

1. BLASTP. 
2. BLASTP using precalculated data. 
3. Clustering.

>> Select a mode or modes (1 or 2 or 1 3 or 2 3): 1
1. AAE
2. CAC
3. ECO
4. ECU
5. HIN
6. LLA
7. SCE
8. SPO
9. SPY
10. SYN
11. TMA
12. YPE
>> Select Genomes to detect Orthologs(e.g. 1 2 3 4 5 or 1-5) : 1
AAE 
1. BLOSUM45
2. BLOSUM62
3. BLOSUM82
4. Quit

Enter a matrix number: 2
You can use 12 processors. 
If you input >2, The Program will run a parallel--Enter the number of CPU you want to use :1


##### if Paramater Passed with Python Code

In [ ]:
##elif sys.argv[1:]
##command_options = argparse_value
#genomes = command_options.genomes
#mode = command_options.mode
#cpu_count= command_options.cpu_count
#blastp_matrix = command_options.blastp_matrix
#inflation_factor = command_options.inflation_factor
#selected_species_dic,backward_selected_species_dic, number_i = Read_Species_List()
#user_selected_number = [backward_selected_species_dic[ele] for ele in genomes]
#Cluster_out = command_options.Cluster_out

In [ ]:
#Species = command_options.Species
#Blastp = COmmand_options.Blastp
#Score_file = command_options.Score_file
#Blastp_data = command_options.Blastp_data
#save_raw_blastp_score = command_options.threshold_score
#threshold_score = command_options.threshold_score
#verbose = command_options.verbose
#infinite_loop = command_options.infinite_loop
#
#if "3" in mode:
#    Check_File(Cluster_out)
#Del_File(score_file, "*")
#if "3" in mode:
#    Log_file_name = Cluster_out+"_S"+str(threshold_score)+"_"+str(inflation_factor)+".log"
#elif not "3" in mode:
#    Log_file_name = "Log"
#
#with open(Log_file_name, "w") as log:
#    log.write(str(datetime.date_time.now()))
#    log.write("\nmode")
#    for i in mode:
#        log.write(" "+i)
#    log.write("\ngenomes:")
#    for i in user_selected_number:
#        log.write(selected_species_dic[i]+" ")
#    log.write("\nCPU_Count:"+str(cpu_count))
#    log.write("\nblastp matrix :"+blastp_matrix)
#    if "3" in mode:
#        log.write("\ninflation_factor: "+str(inflation_factor))
#        log.write("\nCluster out:"+Cluster_out)
#    log.write("\nSpecies :"+Species)
#    log.write("\nBlastp: "+Blastp)
#    log.write("\nScore file : "+ Score_file)
#    log.write("\nBlastp_data : "+ Blastp_data)
#    log.write("\nsave rawblastp score : "+ str(save_raw_blastp_score))
#    log.write("\n")
#

In [ ]:
start_time_OBH = time.time()

In [31]:
if "1" in  mode:
    backward_best_hit_work_list = Oneway_Threshold_Best_Hit(mode)
    pool = multiprocessing.Pool(cpu_count)
    results = pool.map(Backward_Best_Hit, backward_best_hit_work_list)
    pool.close()
    pool.join()
elif "2" in mode:
    used_precalculated_data_list = []
    new_calculated_data_list = []
    precalculated_data_list = glob.glob(Blastp_data+"*oneway_threshold_best_hit_S"+str(threshold_score))
    print(precalculated_data_list)
    backward_best_hit_work_list = Oneway_Threshold_Best_Hit(mode)
    
    if not backward_best_hit_work_list ==[]:
        #If backward_best_hit_work_list is an empty list, pool instance can't finish the work.
        pool = multiprocessing.Pool(cpu_count)
        results = pool.map(Backward_Best_Hit, backward_best_hit_work_list)
        pool.close()
        pool.join()
    else:
        results = [0,0]
Del_File("./","query*")
finish_time_OBH = time.time()
blastp_time_log = float(((finish_time_OBH - start_time_OBH)/60))
print("BLASTP searches + forward best Hit + backwardbest hit took %f minutes"%blastp_time_log)

with open(Log_file_name , "a") as log:
    log.write("Backward_Best_Hit took "+str(max(results))+" minutes\n")
    log.write("BLASTP + Best_Hit + backward_best_hit searches took"+ str(blastp_time_log)+" minutes\n")
if "3" in mode:
    start_time_clustering = time.time()
    #generate matrix calculate the matrix using MCL algorithm and cluster the Ortholog
    
    print("\n >>> Start MCL algorithm and Clustering ortholog <<<")
    equal_BBH_data = []
    unequal_BBH_data = []
    equal_BBH_data_dic = {}
    second_equal_BBH_data = []
    results = que.Queue()
    tasks = queue.Queue()
    cluster_count = 1
    ortholog_count = 0
    gene_id_dic = {}
    
    with open("myva=gb","r") as id_read:
        #myvba=gb is a database
        for i in id_read:
            gene_name,gene_id = i.split()
            gene_id_dic[gene_id.replace("\n","")] = gene_name # remove '\n'
            
        if "1" in mode:
            for i in user_selected_number:
                for k in user_selected_number:
                    if k<i:
                        pass
                    elif i ==k:
                        Read_Equal_BBH(Score_file+selected_species_dic[i]+"_"+selected_species_dic[k])
                    elif i !=k:
                        Read_Unequal_BBH(Score_file+selected_species_dic[i]+"_"+selected_species_dic[k])
        elif "2" in mode:
            for used_data in used_precalculated_data_list:
                first,second = used_data.split("_")
                if first == second:
                    Read_Equal_BBH(Blastp_data+used-data)
                elif first != second:
                    Read_Unequal_BBH(Blastp_data+used_data)
            for new_data in new_calculated_data_list:
                first,second = new_data.split("_")
                if first== second:
                    Read_Equal_BBH(Score_file+new_data)
                elif first != second:
                    Read_Unequal_BBH(Score_file+new_data)
        matched_BBH_data = []
        matched_BBH_element_data_set = []
        
        for unequal_RBH_element in unequal_BBH_data:
            Matching_BBH(unequal_RBH_element)
            temp_results_list = []
            if results._qsize() !=0: #return the number of results as Queue.
                while not results.empty():
                    get_results = results.get()
                    temp_results_list.append(get_results)
                matched_BBH_data.append(temp_results_list)
        #bar = bar("Processing", max= len(matched_BBH_data))
        #bar = bar("Processing", max= len(matched_BBH_data)) #Not Supported in Python 3
        #for data in matched_BBH_data:
        #    Generating_Matrix_Clustering_Ortholog(data, bar)
        #bar.finish()
        finish_time_clustering = time.time()
        mcl_time_log = float((finish_time_clustering - start_time_clustering)/60)
        remark_time_log = float((finish_time_clustering- start_time_OBH)/60)
        print("MCL algorithm and Ortholog Clustering took %.2f minutes"%remark_time_log)
        
        if "3" in mode:
            with open(Log_file_name,"a") as log:
                log.write("Ortholog Count: "+str(ortholog_count)+","+"Cluster count: "+ str(cluster_count-1)+"\n")
                log.write("MCL algorithm and Ortholog Clustering took "+str(mcl_time_log)+"minutes\n")
                log.write("xxx Program took "+str(remark_time_log)+"minutes\n")

NameError: name 'Oneway_Threshold_Best_Hit' is not defined